In [1]:
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="cricket_ml"
)


In [2]:
query = """
SELECT
    g.ground_name,
    COUNT(DISTINCT bd.match_id) AS matches_played,
    SUM(bd.runs_total) AS total_runs
FROM ball_delivery bd
JOIN matches m ON bd.match_id = m.match_id
JOIN ground g ON m.ground_id = g.ground_id
GROUP BY g.ground_name
ORDER BY total_runs DESC
"""
df = pd.read_sql(query, conn)
df


C:\Users\hp\AppData\Local\Temp\ipykernel_28536\3957984024.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,ground_name,matches_played,total_runs
0,Rajiv Gandhi International Stadium,4,1708.0
1,Narendra Modi Stadium,4,1614.0
2,BRSABV Ekana Cricket Stadium,4,1442.0
3,MA Chidambaram Stadium,4,1206.0
4,Eden Gardens,3,1143.0
5,Wankhede Stadium,3,995.0
6,M Chinnaswamy Stadium,3,864.0
7,Arun Jaitley Stadium,2,798.0
8,ACA-VDCA Cricket Stadium,2,749.0
9,Barsapara Cricket Stadium,2,662.0


In [3]:
query = """
SELECT
    g.ground_name,
    SUM(CASE WHEN bd.is_six = 1 THEN 1 ELSE 0 END) AS sixes
FROM ball_delivery bd
JOIN matches m ON bd.match_id = m.match_id
JOIN ground g ON m.ground_id = g.ground_id
GROUP BY g.ground_name
ORDER BY sixes DESC
"""
df = pd.read_sql(query, conn)
df


C:\Users\hp\AppData\Local\Temp\ipykernel_28536\214733606.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,ground_name,sixes
0,Rajiv Gandhi International Stadium,88.0
1,Narendra Modi Stadium,84.0
2,BRSABV Ekana Cricket Stadium,57.0
3,Eden Gardens,56.0
4,Wankhede Stadium,53.0
5,MA Chidambaram Stadium,50.0
6,M Chinnaswamy Stadium,46.0
7,ACA-VDCA Cricket Stadium,45.0
8,Arun Jaitley Stadium,36.0
9,IS Bindra Stadium,33.0


In [4]:
query = """
SELECT
    g.ground_name,
    COUNT(*) AS balls,
    SUM(CASE WHEN bd.is_four = 1 OR bd.is_six = 1 THEN 1 ELSE 0 END) AS boundaries
FROM ball_delivery bd
JOIN matches m ON bd.match_id = m.match_id
JOIN ground g ON m.ground_id = g.ground_id
GROUP BY g.ground_name
HAVING boundaries > 0
"""
df = pd.read_sql(query, conn)

df["balls_per_boundary"] = df["balls"] / df["boundaries"]
df = df.sort_values("balls_per_boundary")
df


C:\Users\hp\AppData\Local\Temp\ipykernel_28536\727653267.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,ground_name,balls,boundaries,balls_per_boundary
7,Rajiv Gandhi International Stadium,968,250.0,3.872000
3,Eden Gardens,715,166.0,4.307229
12,ACA-VDCA Cricket Stadium,463,105.0,4.409524
4,Arun Jaitley Stadium,505,112.0,4.508929
11,Narendra Modi Stadium,1021,211.0,4.838863
1,Wankhede Stadium,677,137.0,4.941606
5,Sawai Mansingh Stadium,233,46.0,5.065217
10,BRSABV Ekana Cricket Stadium,980,193.0,5.077720
2,M Chinnaswamy Stadium,627,116.0,5.405172
8,IS Bindra Stadium,441,78.0,5.653846


In [5]:
query = """
SELECT
    g.ground_name,
    CASE
        WHEN bd.over_number BETWEEN 0 AND 5 THEN 'Powerplay'
        WHEN bd.over_number BETWEEN 6 AND 14 THEN 'Middle'
        ELSE 'Death'
    END AS phase,
    SUM(bd.runs_total) AS runs
FROM ball_delivery bd
JOIN matches m ON bd.match_id = m.match_id
JOIN ground g ON m.ground_id = g.ground_id
GROUP BY g.ground_name, phase
"""
df = pd.read_sql(query, conn)
df


C:\Users\hp\AppData\Local\Temp\ipykernel_28536\4089761523.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,ground_name,phase,runs
0,MA Chidambaram Stadium,Powerplay,399.0
1,MA Chidambaram Stadium,Middle,491.0
2,MA Chidambaram Stadium,Death,316.0
3,Wankhede Stadium,Powerplay,324.0
4,Wankhede Stadium,Middle,448.0
5,Wankhede Stadium,Death,223.0
6,M Chinnaswamy Stadium,Powerplay,262.0
7,M Chinnaswamy Stadium,Middle,408.0
8,M Chinnaswamy Stadium,Death,194.0
9,Eden Gardens,Powerplay,381.0
